In [7]:
from housing_pricer.scraping.sdk.data_manager import DataManager
from housing_pricer.data_processing.data_processing_utils import format_json_to_dataframe, get_nested_dict_value, MissingDataError
import pandas as pd

pd.set_option('display.max_columns', None)
DATA_STORAGE = "../scraping/data_storage"

def lookup_raw_listing(data_manager: DataManager, listing_id: str) -> dict:
    for entry in data_manager.load_data():
        if entry["id"] == listing_id:
            return entry
    raise RuntimeError("Missing entry")

In [11]:
DATA_STORAGE_TEST = "../scraping/data_storage_test"
data_man = DataManager(DATA_STORAGE_TEST)
entry = next(data_man.load_data())
for entry in data_man.load_data():
    print(entry["id"], entry['data']['market_status'])

# with DataManager(DATA_STORAGE_TEST) as data_man:
#     print(data_man._scraped_dates)

annons/5174850 Slutpris
annons/5172906 Slutpris
bostad/720112 Slutpris
bostad/641258 Slutpris
bostad/327104 Slutpris
annons/5189864 Slutpris
bostad/3292728 Slutpris
bostad/3233607 Slutpris
annons/5096431 Slutpris
annons/5162387 Slutpris
annons/5173977 Slutpris
annons/5169445 Slutpris
annons/5136214 Slutpris
bostad/507462 Till salu
bostad/4095240 Slutpris
annons/5162787 Slutpris
annons/5138334 Slutpris
annons/4974864 Slutpris
annons/5119653 Slutpris
bostad/572974 Slutpris
bostad/2671105 Slutpris
annons/5185541 Slutpris
bostad/2706376 Slutpris
annons/5173811 Slutpris
bostad/1716496 Budgivning pågår
annons/5149549 Slutpris
bostad/3952851 Slutpris
bostad/3010375 Slutpris
bostad/496308 Slutpris
bostad/3942579 Slutpris
bostad/163894 Slutpris
bostad/2834576 Slutpris
annons/5162224 Sista bud
annons/5160820 Sista bud
annons/5149145 Sista bud
bostad/1641277 Sista bud
annons/4906478 Sista bud
bostad/4334402 Sista bud
annons/5171054 Sista bud
bostad/3257064 Sista bud
annons/5168199 Sista bud
annon

In [68]:
from housing_pricer.scraping.sdk.scraper import Scraper
from bs4 import BeautifulSoup

data_manager = DataManager("test")
scraper = Scraper(base_url="https://www.booli.se/", data_manager=data_manager, max_requests_per_minute=200, max_delay_seconds=20)
url_id = "annons/5174589"
response = scraper.get(url_id)
parsed_html = BeautifulSoup(response, features="lxml")

In [69]:
from bs4 import BeautifulSoup
def extract_market_status(parsed_html_response: BeautifulSoup, entry_id: str) -> str:
    status = parsed_html_response.find('div', {'class': 'py-1 rounded w-fit bg-bui-color-black text-bui-color-white inline-flex items-center justify-center font-semibold rounded px-2 text-sm'})
    if status:
        return status.text
    raise MissingDataError(f"""Entry with ID: {entry_id} missing market status tag.""")

extract_market_status(parsed_html, url_id)
    



'Sista bud'

In [64]:
from bs4 import BeautifulSoup

parsed_html = BeautifulSoup(response, features="lxml")
status = parsed_html.find('div', {'class': 'py-1 rounded w-fit bg-bui-color-black text-bui-color-white inline-flex items-center justify-center font-semibold rounded px-2 text-sm'})
if status:
    status = status.text
status

'Sista bud'

In [2]:
listings = format_json_to_dataframe(DataManager(DATA_STORAGE).load_data())
listings

Processing scraped JSON content to dataframe...: 838294it [00:37, 22355.56it/s]


,url_listing_type,url_listing_id,booli_id,sold_date,days_listed,residence_type,address,tenure_form,apartment_number,urban_area,municipality,construction_year,list_price,sold_price,sold_price_type,first_price,booli_valuation,booli_valuation_lb,booli_valuation_ub,monthly_payment,rent,operating_cost,energy_class,floor,building_floors,latitude,longitude,has_solar_panels,agency_id,agent_id,booli_ids_of_previous_sales,n_previous_sales
0,bostad,2556516,5153109,2023-12-05,25.0,Villa,Västerbyvägen 9,None,None,Norsholm,Norrköping,1979.0,3325000.0,3250000.0,Slutpris,3 325 000,NaN,NaN,NaN,None,NaN,4398.0,D,None,NaN,58.505988,15.976788,None,26,11016,[5153109],1
1,bostad,1890105,5147734,2023-12-05,32.0,Villa,Sliparevägen 3,None,None,Rejmyre,Finspång,1978.0,1495000.0,1495000.0,Slutpris,1 495 000,NaN,NaN,NaN,None,NaN,4102.0,C,None,NaN,58.826413,15.938500,None,962,9543,"[5147734, -9035770]",2
2,bostad,1706785,5168064,2023-12-05,8.0,Villa,Kronoparksvägen 22,None,None,Ivetofta,Bromölla,1980.0,2095000.0,2400000.0,Slutpris,2 095 000,NaN,NaN,NaN,None,NaN,2648.0,C,None,NaN,56.091915,14.466163,None,20,11931,"[5168064, 984971, 350836]",3
3,bostad,3997519,5078126,2023-12-05,102.0,Lägenhet,Borgarfjordsgatan 21B,None,1303,Kista,Stockholm,2015.0,2495000.0,2450000.0,Slutpris,2 595 000,NaN,NaN,NaN,None,4274.0,NaN,D,3,NaN,59.404946,17.941505,None,16,6529,[5078126],1
4,annons,5119460,5119460,2023-12-05,39.0,Lägenhet,Rålambsvägen 48,None,None,Fredhäll,Stockholm,1931.0,3400000.0,3400000.0,Slutpris,3 195 000,NaN,NaN,NaN,None,2213.0,855.0,None,3,NaN,59.330127,18.005056,None,34,9162,[5119460],1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
838289,bostad,2226597,2226597,None,NaN,Radhus,Kantorsgränd 20,None,None,Sollentuna kommun,None,1979.0,NaN,NaN,None,None,5240000.0,4880000.0,5610000.0,None,NaN,3458.0,None,None,NaN,59.464725,17.919206,None,None,None,[3450322],1
838290,annons,3570926,3570926,None,NaN,Lägenhet,Stormgatan 6K,None,None,Nya Kungsgården,Ängelholm,1991.0,NaN,NaN,None,None,NaN,NaN,NaN,None,6301.0,400.0,None,1,NaN,56.243973,12.891679,None,None,None,[3570926],1
838291,bostad,3544356,3544356,None,NaN,Villa,Hössjö 232,None,None,Umeå kommun,None,1983.0,NaN,NaN,None,None,2920000.0,2650000.0,3200000.0,None,NaN,1823.0,None,None,NaN,63.785965,19.865358,None,None,None,[3551987],1
838292,bostad,2023799,2023799,None,NaN,Kedjehus,Åkre 1:56,None,None,Östersunds kommun,None,1992.0,NaN,NaN,None,None,1920000.0,1650000.0,2190000.0,None,NaN,2401.0,None,None,NaN,63.084201,14.830180,None,None,None,"[3525891, -7344427]",2


In [59]:
listings.query("sold_price.isna()").head(30)

,url_listing_type,url_listing_id,booli_id,sold_date,days_listed,residence_type,address,tenure_form,apartment_number,urban_area,municipality,construction_year,list_price,sold_price,sold_price_type,first_price,booli_valuation,booli_valuation_lb,booli_valuation_ub,monthly_payment,rent,operating_cost,energy_class,floor,building_floors,latitude,longitude,has_solar_panels,agency_id,agent_id,booli_ids_of_previous_sales,n_previous_sales
2330,bostad,1804068,5179391,None,4.0,Villa,Näckrosvägen 16,Äganderätt,None,Kallhäll,Järfälla,1962.0,6950000.0,NaN,None,None,6930000.0,6430000.0,7440000.0,17 992 kr/mån,NaN,3883.0,D,None,NaN,59.467026,17.833508,False,12,1287,"[5060887, 1385395]",2
3218,bostad,4054768,4465595,None,735.0,Lägenhet,Ebbe Lieberathsgatan 16C,Bostadsrätt,1310,Södra Centrum,Göteborg,2018.0,NaN,NaN,None,None,2290000.0,2180000.0,2410000.0,None,2438.0,NaN,C,4,8.0,57.680065,12.001776,False,205,10488,"[5175171, 3649693]",2
3340,bostad,3123818,5174381,None,12.0,Villa,Lärkvägen 14,Äganderätt,None,Charlottenberg,Eda,1964.0,1750000.0,NaN,None,None,1870000.0,1460000.0,2290000.0,5 872 kr/mån,NaN,1933.0,B,None,NaN,59.890129,12.304524,False,854,10543,"[5030805, 4369342]",2
3624,bostad,4043820,5173866,None,13.0,Lägenhet,Torsplan 8,Bostadsrätt,1901,Vasastan,Stockholm,2018.0,NaN,NaN,None,None,6790000.0,6460000.0,7120000.0,None,NaN,NaN,D,9,36.0,59.346026,18.033846,False,952,9021,"[4997836, 4030233, 3682004]",3
7314,bostad,4100675,5171993,None,4.0,Lägenhet,Blåklockevägen 13,Bostadsrätt,1501,Tureberg,Sollentuna,2016.0,3595000.0,NaN,None,None,3450000.0,3280000.0,3630000.0,13 447 kr/mån,5915.0,650.0,D,6,7.0,59.433223,17.942997,False,12,12930,[5034061],1
7714,bostad,653941,5179000,None,6.0,Lägenhet,Legendvägen 23,Bostadsrätt,1203,Kallhäll,Järfälla,1964.0,1975000.0,NaN,None,None,1830000.0,1730000.0,1930000.0,10 827 kr/mån,6296.0,NaN,E,3,3.0,59.452465,17.816153,False,32,13568,"[5043421, 3075943]",2
10033,bostad,503080,5171013,None,16.0,Lägenhet,Värslevägen 61,Bostadsrätt,1002,Askim,Göteborg,1965.0,1695000.0,NaN,None,None,1680000.0,1600000.0,1770000.0,7 029 kr/mån,3034.0,NaN,D,BV,NaN,57.632670,11.933369,False,12,6914,"[5063977, 4679221]",2
10095,bostad,2482450,5159682,None,29.0,Villa,Idekullavägen 29,Äganderätt,None,Ryd,Tingsryd,1975.0,1195000.0,NaN,None,None,1220000.0,919000.0,1530000.0,7 479 kr/mån,NaN,4631.0,E,None,NaN,56.456896,14.703811,False,34,13391,[4934200],1
10102,bostad,2618222,5176691,None,8.0,Villa,Opalvägen 54,Äganderätt,None,Hyllinge,Åstorp,1975.0,NaN,NaN,None,None,2890000.0,2470000.0,3310000.0,None,NaN,4042.0,D,None,NaN,56.104167,12.877018,False,32,13658,"[5053052, 2282558]",2
10138,bostad,2880219,4961213,None,229.0,Villa,Bävergiljan 4,Äganderätt,None,Bohuslän,Uddevalla,1928.0,1995000.0,NaN,None,None,1830000.0,1560000.0,2090000.0,8 408 kr/mån,NaN,NaN,C,None,NaN,58.363432,11.938819,False,20,11420,[5059375],1


In [21]:
bostad = lookup_raw_listing(data_manager=DataManager(DATA_STORAGE), listing_id="bostad/2226597")
bostad

{'id': 'bostad/2226597',
 'data': {'Area_V3:2': {'__typename': 'Area_V3',
   'id': '2',
   'name': 'Stockholms län',
   'parent': '77104',
   'type': 'county'},
  'Area_V3:13': {'__typename': 'Area_V3',
   'id': '13',
   'name': 'Sollentuna',
   'parent': '2',
   'type': 'municipality'},
  'Area_V3:1322': {'__typename': 'Area_V3',
   'id': '1322',
   'name': 'Norrviken',
   'parent': '13',
   'type': 'userDefined'},
  'Area_V3:77104': {'__typename': 'Area_V3',
   'id': '77104',
   'name': 'Sverige',
   'parent': 'undefined',
   'type': 'country'},
  'Area_V3:103542': {'__typename': 'Area_V3',
   'id': '103542',
   'name': 'Kantorsgränd',
   'parent': '13',
   'type': 'streetName'},
  'Area_V3:859388': {'__typename': 'Area_V3',
   'id': '859388',
   'name': '19272',
   'parent': '851993',
   'type': 'postcode'},
  'Area_V3:914053': {'__typename': 'Area_V3',
   'id': '914053',
   'name': 'Norrort',
   'parent': '2',
   'type': 'userDefined'},
  'Area_V3:973576': {'__typename': 'Area_V3',

In [ ]:
bostad

# Data analysis

In [20]:
import plotly.express as px
viable_listings = listings.query("sold_date.notna()").copy()
px.scatter(x=viable_listings.sold_date.unique(), y=viable_listings.sold_date.value_counts())

In [ ]:
viable_listings["sold_YYYY_MM"] = [date[:-3] for date in viable_listings["sold_date"]]

viable_listings

,url_listing_type,url_listing_id,booli_id,sold_date,days_listed,residence_type,address,apartment_number,urban_area,municipality,construction_year,list_price,sold_price,sold_price_type,first_price,monthly_payment,rent,operating_cost,energy_class,floor,building_floors,latitude,longitude,has_solar_panels,agency_id,agent_id,booli_ids_of_previous_sales,n_previous_sales,sold_YYYY_MM
0,bostad,2556516,5153109,2023-12-05,25.0,Villa,Västerbyvägen 9,None,Norsholm,Norrköping,1979.0,3325000.0,3250000.0,Slutpris,3 325 000,None,NaN,4398.0,D,None,NaN,58.505988,15.976788,None,26,11016,[5153109],1,2023-12
1,bostad,1890105,5147734,2023-12-05,32.0,Villa,Sliparevägen 3,None,Rejmyre,Finspång,1978.0,1495000.0,1495000.0,Slutpris,1 495 000,None,NaN,4102.0,C,None,NaN,58.826413,15.938500,None,962,9543,"[5147734, -9035770]",2,2023-12
2,bostad,1706785,5168064,2023-12-05,8.0,Villa,Kronoparksvägen 22,None,Ivetofta,Bromölla,1980.0,2095000.0,2400000.0,Slutpris,2 095 000,None,NaN,2648.0,C,None,NaN,56.091915,14.466163,None,20,11931,"[5168064, 984971, 350836]",3,2023-12
3,bostad,3997519,5078126,2023-12-05,102.0,Lägenhet,Borgarfjordsgatan 21B,1303,Kista,Stockholm,2015.0,2495000.0,2450000.0,Slutpris,2 595 000,None,4274.0,NaN,D,3,NaN,59.404946,17.941505,None,16,6529,[5078126],1,2023-12
4,annons,5119460,5119460,2023-12-05,39.0,Lägenhet,Rålambsvägen 48,None,Fredhäll,Stockholm,1931.0,3400000.0,3400000.0,Slutpris,3 195 000,None,2213.0,855.0,None,3,NaN,59.330127,18.005056,None,34,9162,[5119460],1,2023-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309957,bostad,2407266,4818529,2023-06-01,133.0,Villa,Zacharias väg 14,None,Kulltorp,Kristianstad,1937.0,2695000.0,2630000.0,Lagfart,2 890 000,None,NaN,2373.0,G,None,NaN,56.052645,14.185546,None,1071,11246,"[4818529, 4547164]",2,2023-06
310141,bostad,518000,4990861,2023-06-01,10.0,Lägenhet,Norra Liden 8,1501,Kungshöjd Och Inom Vallgraven,Göteborg,1947.0,5850000.0,5900000.0,Slutpris,5 850 000,None,4575.0,325.0,D,4,NaN,57.704597,11.958324,None,294,11605,"[4990861, 4577769]",2,2023-06
310187,bostad,4010918,5111124,2023-12-05,32.0,Lägenhet,Råcksta Gårdsväg 28,1802,Vällingby Parkstad,Stockholm,2014.0,2495000.0,2650000.0,Slutpris,2 495 000,None,3583.0,375.0,B,8,NaN,59.355201,17.885914,None,32,10216,"[5111124, 4581935]",2,2023-12
310450,bostad,441268,5107654,2023-12-14,61.0,Lägenhet,Karolinagatan 1,1101,Huvudsta,Solna,1951.0,4995000.0,5125000.0,Sista bud,5 395 000,None,5189.0,1146.0,G,None,NaN,59.352290,17.996105,None,12,13693,"[5107654, 4570371]",2,2023-12
